In [4]:
import sys
sys.path.append('..')
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)


# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=3500,
                        temperature=0.0)
# for key, value in os.environ.items():
#   print(f"{key}: {value}")

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [5]:
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        # to_json()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in fetching metadata from cube: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore


In [7]:
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"


Vector store loaded successfully!


In [8]:
len(docs)

TypeError: object of type 'FAISS' has no len()

In [6]:
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
cube_metadata_documents = fetch_cube_metadata()

Loading metadata from http://localhost:4000/cubejs-api/v1...
Error in fetching metadata from cube: HTTPConnectionPool(host='localhost', port=4000): Max retries exceeded with url: /cubejs-api/v1/meta (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12f4368d0>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [3]:

loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
cube_metadata = loader.fetch_raw_cube_metadata()

Loading metadata from http://localhost:4000/cubejs-api/v1...


In [ ]:
import json

filepath = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/preloaded_cube_metadata/cube_metadata.json"
#save the cube metadata to a file
with open(filepath, "w") as file:
    json.dump(cube_metadata, file)

print("Cube metadata saved successfully!")

In [ ]:
#load the cube metadata from the file and format the data into list[Document] for further processing while using 
import json

filepath = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/preloaded_cube_metadata/cube_metadata.json"

# Load the JSON data from the file
with open(filepath, "r") as file:
    data = json.load(file)
documents = loader.format_raw_cube_metadata(data)

In [4]:
vectorstore = load_vector_store(vector_store_path, embeddings)

Vector store loaded successfully!


In [ ]:
#Controller
from langchain import PromptTemplate
start_goal_prompt = PromptTemplate(
    template="""
    You are an very experienced and expert manager. You are given following query "{goal}".
    The query may contain different objectives which needs an expertise to solve. Your job is to provide a list of objectives that is easier for the experts to serve the query.
    Ensure objectives are streamlined to minimize the overall number of tasks while addressing the objective effectively. 
    Some part of the query may not be relevant to the objectives. You can ignore those parts.
    The number of tasks should be strictly less than 10.
    """,
    input_variables=["goal", "language", "tools"],
)

Tools

Metric Discovery: Assuming we are trying to 3 metrics 
Agent: Get relevant 
Get 3 top documents 

Tasks: Metric Discovery 
1. Rewrite the and Extract specifics of metric information for better retrieval
2. Extract top 3 documents related to metric information. (Basic RAG)
3. get information about metrics and compile the results and return 

Tasks: Information regarding the metric
1. Check if metric exists
2. Query vector database to descrive about the metric (RAG)



In [5]:
import json
from crewai import Agent, Task
from langchain.tools import tool
from src.utils.llamaindex_retriever import LlamaIndexRetriever
#Will create below functions once the input is setup properly
# def create_vector_store

# def ingest_cube_meta():



@tool("Extract metric queried.")
def extract_metric_description(query, *args, **kwargs):
  """Useful to isolate metric information to be further used in the its discovery"""
  agent = Agent(
        role='Metric Isolator',
        goal=
        'Accurate Extraction of Metric name or description',
        backstory=
        "You're a Metric isolator which is responsible for extracting the metrics name from the given content.",
        llm = llm_gpt,
        allow_delegation=False)
  task = Task(
        agent=agent,
        description=
        f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
        expected_output="some string",
  
    )
  extracted_metrics = task.execute()

  return extracted_metrics

faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

def get_similar_documents(query, max_number_documents=3):
  faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
  vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
  docs = vectorstore.similarity_search_with_relevance_scores(query, max_number_documents)
  conversational_metadata = []
  for doc in docs:
      doc = doc[0]
      meta = {'text':doc.page_content, 'table_metadata': doc.metadata}
      conversational_metadata.append(meta)
  return conversational_metadata

# @tool("Metric Discovery.")
def metric_discovery_faiss(metric_description: int, *args, **kwargs):
  """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
  # documents = fetch_cube_metadata()
  # retriever = LlamaIndexRetriever('metric discovery')
  # retriever.index_document(documents)

  # docs = vectorstore.similarity_search_with_relevance_scores(metric_description, max_number_documents)
  # conversational_metadata = []
  # for doc in docs:
  #   meta = {'text':doc.text, 'table_metadata': doc.metadata}
  #   conversational_metadata.append(meta)
  conversational_metadata = get_similar_documents(metric_description)
  agent = Agent(
        role='Data Analyst Assistant',
        goal=
        'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
        backstory=
        "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
        llm = llm_gpt,
        allow_delegation=False)  
  
  task = Task(
        agent=agent,
        description=
        """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {conversational_metadata}}.
            Strictly answer the question with the information present in metadata only.
            Respond with "Sorry, the query is out of scope." if the answer is not present in metadata.
            """,
        expected_output="some string",
  
    )
  output = task.execute()

  return output

@tool("capture all possible user intent .")
def capture_query_intent_possibilities(query, *args, **kwargs):
  """Useful to capture the intent of the user regarding metric information"""
  agent = Agent(
        role='Intent Capturer',
        goal=
        'capture the intent of the user regarding metric information',
        backstory=
        "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
        llm = llm_gpt,
        allow_delegation=False)
  task = Task(
        agent=agent,
        description=
        f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
        expected_output="some string",
  
    )
  extracted_metrics = task.execute()

  return extracted_metrics

@tool("Rephrase input query.")
def rephrase_input_query(query, *args, **kwargs):
  """Useful to rephrase the query to capture the intent of the user regarding metric information"""
  agent = Agent(
        role='Intent Capturer',
        goal=
        'Rephrasing the query to capture the intent of the user regarding metric information',
        backstory=
        "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
        llm = llm_gpt,
        allow_delegation=False)
  task = Task(
        agent=agent,
        description=
        f'Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response',
        expected_output="some string",
  
    )
  extracted_metrics = task.execute()

  return extracted_metrics


Response of capture_query_intent_possibilities tool
To accurately capture the user's intent regarding metric information for the query "What is the most popular feature used by our paid subscribers," the following list isolates and extracts the key metrics:

1. Feature popularity ranking among paid subscribers
2. Usage statistics of features by paid subscribers
3. Comparison of feature usage between paid and non-paid subscribers (if applicable)
4. Identification of the single most used feature by paid subscribers

In [12]:
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
class MetricDiscoveryInput(BaseModel):
    # model_config = ConfigDict(arbitrary_types_allowed=True)
    metric_description: str = Field(description="query regarding metric")

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = MetricDiscoveryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        output = metric_discovery_faiss(query)
        return output

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [8]:
# from langchain.tools import BaseTool, StructuredTool, tool
# from langchain.pydantic_v1 import BaseModel, Field
# # from langchain.pydantic_v1 import  Field
# # from pydantic import BaseModel
# from typing import Optional

# from pydantic import BaseModel, ConfigDict, ValidationError

# class MetricDiscoveryInput(BaseModel):
#     # model_config = ConfigDict(arbitrary_types_allowed=True)
#     metric_description: str = Field(description="query regarding metric")
#     # vectorstore: Optional[FAISS] = Field(description="vectorstore to get indexed documents")

# metric_discovery = StructuredTool.from_function(
#     func=metric_discovery_faiss,
#     name="Metric Discovery.",
#     description="""Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.""",
#     args_schema=MetricDiscoveryInput,
#     return_direct=True,
# )

ValidationError: 1 validation error for StructuredTool
args_schema
  subclass of BaseModel expected (type=type_error.subclass; expected_class=BaseModel)

prompt_metric_discovery = f"You are given a metric description as input {metric_description}. Your task is to determine whether this specific metric exists within the context provided. Thoroughly analyze the metric description and provide a definitive answer on the existence of the metric, along with any relevant details or insights that support your conclusion."

In [18]:
faiss_index_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
query = "What is the most popular feature used by our paid subscribers"
# response = metric_discovery_faiss(query, vectorstore)

In [13]:
query = "What is the most popular feature used by our paid subscribers"
dummy_tool = CustomSearchTool()
response = dummy_tool.run(query)
print(response)

Sorry, the query is out of scope.


In [57]:
from crewai import Task
from textwrap import dedent

class MetricDiscoveryTasks():
  def metric_isolation(self, agent, query):
    return Task(description=(f"""
        Given query {query}, extract the description of the metric precisely.
        Rewrite the description such that the it is more understandable to the common user.
        
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="some string",
    )
  def __tip_section(self):
    return "If you do your BEST WORK, I'll give you a $10,000 commission!"
  
   


In [58]:
query ="Helo, I am looking for the most popular feature used by our paid subscribers"
print(f"""
        Given query {query}, extract the description of the metric precisely.
        Rewrite the description such that the it is more understandable to the common user.
        
        """)


        Given query Helo, I am looking for the most popular feature used by our paid subscribers, extract the description of the metric precisely.
        Rewrite the description such that the it is more understandable to the common user.
        
        


In [81]:
from crewai import Agent

class MetricDiscoveryAgent(Agent):
  def metric_isolation(self):
    return Agent(
      role="Extract user intent",
      goal=dedent("""{Capture metric information from the user query}"""),
      backstory=dedent("""{You are an expert to understand and interpret what the user is asking for. You need to extract the metric information from the user query and provide a more understandable version of the metric description.}"""),
      verbose=True,
      tools=[
        extract_metric_description()
        
      ],
      llm = llm_gpt,
    )


In [83]:
agent = Agent(
      role="Extract user intent",
      goal=dedent("""{Capture metric information from the user query}"""),
      backstory=dedent("""{You are an expert to understand and interpret what the user is asking for. You need to extract the metric information from the user query and provide a more understandable version of the metric description.}"""),
      verbose=True,
      tools=[
        extract_metric_description
      ],
      llm = llm_gpt,
    )




In [84]:
query ="'Helo, I am looking for the most popular feature used by our paid subscribers'"
tasks = MetricDiscoveryTasks()
task = tasks.metric_isolation(agent, query)
print(task)

used_tools=0 tools_errors=0 delegations=0 i18n=I18N(prompt_file=None) thread=None prompt_context=None description="\n        Given query 'Helo, I am looking for the most popular feature used by our paid subscribers', extract the description of the metric precisely.\n        Rewrite the description such that the it is more understandable to the common user.\n        \n        If you do your BEST WORK, I'll give you a $10,000 commission!" expected_output='some string' config=None callback=None agent=Agent(role=Extract user intent, goal={Capture metric information from the user query}, backstory={You are an expert to understand and interpret what the user is asking for. You need to extract the metric information from the user query and provide a more understandable version of the metric description.}) context=None async_execution=False output_json=None output_pydantic=None output_file=None output=None tools=[StructuredTool(name='Extract metric queried.', description='Useful to isolate met

In [87]:
from crewai import Crew
class MetricDiscoveryInputCrew:
  def __init__(self, query):
    self.query = query
    # self.llm = llm_gpt

  def run(self):
    # agents = MetricDiscoveryAgent()
    tasks = MetricDiscoveryTasks()
    # print(agents)
    print(tasks)
    # metric_isolator_agent = agents.metric_isolation()
    # print(metric_isolator_agent)
    # metric_isolator_task = tasks.metric_isolation(metric_isolator_agent, self.query)
    metric_isolator_task = tasks.metric_isolation(agent, self.query)
    
    print("Metric_isolator_task", metric_isolator_task)
    crew = Crew(
      agents=[
        # metric_isolator_agent,
        agent
      ],
      tasks=[
        metric_isolator_task
      ],
      verbose=False
    )

    result = crew.kickoff()
    return result

In [88]:
query = "What is the most popular feature used by our paid subscribers"
# formatted_query = input(
#     dedent("""
#       {What is the most popular feature used by our paid subscribers}
#     """))
# print(formatted_query)
crew = MetricDiscoveryInputCrew(query)
result = crew.run()

2024-05-30 11:31:02,746 - 140704506453952 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Metric_isolator_task used_tools=0 tools_errors=0 delegations=0 i18n=I18N(prompt_file=None) thread=None prompt_context=None description="\n        Given query What is the most popular feature used by our paid subscribers, extract the description of the metric precisely.\n        Rewrite the description such that the it is more understandable to the common user.\n        \n        If you do your BEST WORK, I'll give you a $10,000 commission!" expected_output='some string' config=None callback=None agent=Agent(role=Extract user intent, goal={Capture metric information from the user query}, backstory={You are an expert to understand and interpret what the user is asking for. You need to extract the metric information from the user query and provide a more understandable version of the metric description.}) context=None async_execution=False output_json=None output_pydantic=None output_file=None output=None tools=[StructuredTool(name='Extract metric queried.', description='Useful to isolate

In [71]:
type(formatted_query)


print(formatted_query)

In [33]:
formtted_query

''

In [22]:
docs = vectorstore.similarity_search("What is the most popular feature used by our paid subscribers", 3)

In [ ]:
@tool("Metric Discovery.")
def metric_discovery_faiss(metric_description, *args, **kwargs):
  """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
  documents = fetch_cube_metadata()
  retriever = LlamaIndexRetriever('metric discovery')
  retriever.index_document(documents)

  # rephrased_query = f"Find the metric {metric_description}"

  response_doc = retriever.query_util(metric_description, documents)
  conversational_metadata = []
  for doc in response_doc:
    meta = {'text':doc.text, 'table_metadata': doc.metadata}
    conversational_metadata.append(meta)
  agent = Agent(
        role='Data Analyst Assistant',
        goal=
        'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
        backstory=
        "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
        llm = llm_gpt,
        allow_delegation=False)  
  
  task = Task(
        agent=agent,
        description=
        """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {conversational_metadata}}.
            Strictly answer the question with the information present in metadata only.
            Respond with "Sorry, the query is out of scope." if the answer is not present in metadata.
            """,
        expected_output="some string",
  
    )
  output = task.execute()

  return output

In [ ]:
query = "What is the most popular feature used by our paid subscribers"
extracted_metrics = extract_metric_description(query)

In [ ]:
extracted_metrics

In [ ]:
def get_similar_documents(query, vector_store):
  docs = vector_store.similarity_search(query)
  # take the first document as the best guess
  table_name = docs[0].metadata["table_name"]
  # Columns
  columns_question = "All available columns"
  column_docs = vector_store.similarity_search(
      columns_question,
      filter=dict(table_name=table_name),
      k=20
  )
  lines = []
  for column_doc in column_docs:
      column_title = column_doc.metadata["column_title"]
      column_name = column_doc.metadata["column_name"]
      column_data_type = column_doc.metadata["column_data_type"]
      lines.append(
          f"title: {column_title}, column name: {column_name}, datatype: {column_data_type}, member type: {column_doc.metadata['column_member_type']}"
      )
  columns = "\n".join(lines)
  print("columns :", columns)

  FAISS.similarity_search_with_relevance_scores

In [ ]:
%pip install nltk

In [ ]:
# pre process of input to make it more relevant
import nltk

# Download NLTK tokenizer data (one-time setup)
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
  # Tokenization
  tokens = nltk.word_tokenize(text.lower())
  # Stop word removal
  stopwords = nltk.corpus.stopwords.words('english')
  tokens = [token for token in tokens if token not in stopwords]
  return tokens

def reformat_question(question):
  # Define synonyms and related terms
  metrics = ["metric", "KPI", "measure"]
  sales_terms = ["sales", "revenue", "turnover"]
  central_tendency = ["average", "mean", "median"]
  
  # Preprocess the question
  preprocessed_question = preprocess_text(question)
  
  # Build the reformulated query string using question tokens
  query_string = "(" + " OR ".join(metrics) + ")"
  query_string += " AND (" + " OR ".join(sales_terms) + ")"
  query_string += " AND (" + " OR ".join(central_tendency) + ")"
  
  # Include question tokens for broader coverage
  query_string += " AND (" + " OR ".join(preprocessed_question) + ")"
  
  return query_string

# Example usage
question = "Get the metric for sales on avg"
reformulated_question = reformat_question(question)
print(f"Original question: {question}")
print(f"Reformatted question for FAISS search: {reformulated_question}")


In [ ]:
from src.utils.langchain_retriever import LangchainRetriever
langchain_retriever = LangchainRetriever(embeddings)

In [ ]:
documents = fetch_cube_metadata()
langchain_retriever.index_documents(documents)

In [ ]:
type(documents[0])

In [ ]:
subdocs = langchain_retriever.retrieve_sub_document()

In [1]:
from langchain_community.document_loaders import TextLoader

loaders = [
    TextLoader("/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file1.txt"),
    TextLoader("/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file2.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
docs[0]

Document(page_content="---\n\n**Shipping and Logistics Company Metrics and KPIs**\n\n1. **Order Accuracy Rate**\n   - **Description**: Measures the percentage of orders correctly processed and delivered without errors.\n   - **Formula**: (Correct Orders / Total Orders) x 100\n   - **Importance**: High accuracy in order processing reduces returns and increases customer satisfaction.\n\n2. **On-Time Delivery Rate**\n   - **Description**: Indicates the percentage of shipments delivered on or before the promised delivery date.\n   - **Formula**: (On-Time Deliveries / Total Deliveries) x 100\n   - **Importance**: Timely deliveries enhance customer trust and loyalty.\n\n3. **Average Delivery Time**\n   - **Description**: Calculates the average time taken from order placement to delivery.\n   - **Formula**: Total Delivery Time / Number of Deliveries\n   - **Importance**: Helps in assessing the efficiency of the delivery process and identifying areas for improvement.\n\n4. **Transportation Cos

In [ ]:
langchain_retriever.index_documents(docs)

In [ ]:
subdocs = langchain_retriever.retrieve_sub_document("vectorstore")

In [ ]:
# Uncomment the following line if you need to initialize FAISS with no AVX2 optimization
# os.environ['FAISS_NO_AVX2'] = '1'

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

import glob, os
input_folder_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files"
txt_files = glob.glob(os.path.join(input_folder_path, "*.txt"))

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

In [1]:
import glob, os
input_folder_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files"
txt_files = glob.glob(os.path.join(input_folder_path, "*.txt"))
print(txt_files)

['/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file2.txt', '/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/dummy_txt_files/file1.txt']


In [3]:
from langchain_community.document_loaders import TextLoader
documents = []
for txt_file in txt_files:
    loader = TextLoader(txt_file)
    documents.extend(loader.load())

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

2
